In [1]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [4]:
L=8064
Fs=128
Features=np.zeros((25600,32,3*128))
Labels=list()
for subject_id in range(0,32):
    if subject_id % 10 ==0:
      print(subject_id)
    loaded_data=None
    file_name=None
    if subject_id<9:
        file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
    else:
        file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        loaded_data = u.load()
    data=loaded_data['data']
    Labels.append(loaded_data['labels'])
    
    for i in range(len(data)):
        for k in range(3,61,3):
          for j in range(32):   
            x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
            x=zscore(x)
            Features[subject_id*800+i*20+int(k/3-1)][j]=x

0
10
20
30


In [5]:

def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)



In [6]:
y=np.zeros(800*32)
for j in range(32):
  for i in range(40):
    if Labels[j][i][0]>5:
      y[j*800+i*20:j*800+(i+1)*20]=1

In [7]:
def run(train_index,test_index,
        dropout_rate=0.5,F1 = 8,D = 2, F2 = 16):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  


  model = EEGNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate, kernLength = 32, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')
 

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =30 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [8]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
train_index=list(kf.split(Features))[0][0]
test_index=list(kf.split(Features))[0][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 42s - loss: 0.7564 - accuracy: 0.5467 - val_loss: 0.6850 - val_accuracy: 0.5531

Epoch 00001: val_loss improved from inf to 0.68502, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 12s - loss: 0.7348 - accuracy: 0.5921 - val_loss: 0.6661 - val_accuracy: 0.5992

Epoch 00002: val_loss improved from 0.68502 to 0.66607, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7288 - accuracy: 0.6046 - val_loss: 0.6470 - val_accuracy: 0.6225

Epoch 00003: val_loss improved from 0.66607 to 0.64700, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 12s - loss: 0.7195 - accuracy: 0.6168 - val_loss: 0.6509 - val_accuracy: 0.6176

Epoch 00004: val_loss did not improve from 0.64700
Epoch 5/30
320/320 - 12s - loss: 0.7127 - accuracy: 0.6264 - val_loss: 0.6447 - val_accuracy: 0.6205

Epoch 00005: val_loss improved from 0.64700 to 0.64473, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 12s - loss: 0.7097 - accuracy: 0.6271 - val_loss: 0.6381 -

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 0, 0, ..., 0, 0, 0]),
 0.6939453125,
 0.7041721729280725,
 array([[1688,  528],
        [1039, 1865]]),
 <keras.engine.functional.Functional at 0x7ff680337f10>)

In [ ]:
train_index=list(kf.split(Features))[1][0]
test_index=list(kf.split(Features))[1][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 13s - loss: 0.7580 - accuracy: 0.5529 - val_loss: 0.6721 - val_accuracy: 0.5949

Epoch 00001: val_loss improved from inf to 0.67208, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7418 - accuracy: 0.5920 - val_loss: 0.6581 - val_accuracy: 0.6197

Epoch 00002: val_loss improved from 0.67208 to 0.65808, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7348 - accuracy: 0.6031 - val_loss: 0.6533 - val_accuracy: 0.6213

Epoch 00003: val_loss improved from 0.65808 to 0.65329, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7256 - accuracy: 0.6159 - val_loss: 0.6431 - val_accuracy: 0.6391

Epoch 00004: val_loss improved from 0.65329 to 0.64310, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7197 - accuracy: 0.6177 - val_loss: 0.6363 - val_accuracy: 0.6383

Epoch 00005: val_loss improved from 0.64310 to 0.63627, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 11s - loss: 0.7168

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([0, 1, 1, ..., 0, 0, 0]),
 0.7115234375,
 0.735162273623812,
 array([[1593,  714],
        [ 763, 2050]]),
 <keras.engine.functional.Functional at 0x7ff68032a7d0>)

In [ ]:
train_index=list(kf.split(Features))[2][0]
test_index=list(kf.split(Features))[2][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 14s - loss: 0.7615 - accuracy: 0.5341 - val_loss: 0.6763 - val_accuracy: 0.5650

Epoch 00001: val_loss improved from inf to 0.67628, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7415 - accuracy: 0.5841 - val_loss: 0.6563 - val_accuracy: 0.5996

Epoch 00002: val_loss improved from 0.67628 to 0.65632, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7307 - accuracy: 0.6026 - val_loss: 0.6424 - val_accuracy: 0.6178

Epoch 00003: val_loss improved from 0.65632 to 0.64239, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7225 - accuracy: 0.6124 - val_loss: 0.6411 - val_accuracy: 0.6240

Epoch 00004: val_loss improved from 0.64239 to 0.64106, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7161 - accuracy: 0.6198 - val_loss: 0.6376 - val_accuracy: 0.6498

Epoch 00005: val_loss improved from 0.64106 to 0.63758, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 11s - loss: 0.7104

In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 1, 1]),
 array([1, 1, 1, ..., 1, 0, 0]),
 0.7181640625,
 0.7358594179022516,
 array([[1667,  608],
        [ 835, 2010]]),
 <keras.engine.functional.Functional at 0x7ff64594b610>)

In [9]:
train_index=list(kf.split(Features))[3][0]
test_index=list(kf.split(Features))[3][1]

In [10]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 41s - loss: 0.7561 - accuracy: 0.5547 - val_loss: 0.6819 - val_accuracy: 0.5561

Epoch 00001: val_loss improved from inf to 0.68186, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7342 - accuracy: 0.5959 - val_loss: 0.6553 - val_accuracy: 0.6092

Epoch 00002: val_loss improved from 0.68186 to 0.65535, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7244 - accuracy: 0.6111 - val_loss: 0.6468 - val_accuracy: 0.6326

Epoch 00003: val_loss improved from 0.65535 to 0.64682, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7153 - accuracy: 0.6277 - val_loss: 0.6344 - val_accuracy: 0.6398

Epoch 00004: val_loss improved from 0.64682 to 0.63442, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7067 - accuracy: 0.6347 - val_loss: 0.6287 - val_accuracy: 0.6578

Epoch 00005: val_loss improved from 0.63442 to 0.62873, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 12s - loss: 0.6981

In [11]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 0, 0, ..., 0, 0, 0]),
 0.7140625,
 0.7122641509433962,
 array([[1844,  454],
        [1010, 1812]]),
 <keras.engine.functional.Functional at 0x7f2580337890>)

In [12]:
train_index=list(kf.split(Features))[4][0]
test_index=list(kf.split(Features))[4][1]

In [13]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 13s - loss: 0.7598 - accuracy: 0.5455 - val_loss: 0.6753 - val_accuracy: 0.5941

Epoch 00001: val_loss improved from inf to 0.67529, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 11s - loss: 0.7415 - accuracy: 0.5896 - val_loss: 0.6592 - val_accuracy: 0.6070

Epoch 00002: val_loss improved from 0.67529 to 0.65919, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 11s - loss: 0.7297 - accuracy: 0.6035 - val_loss: 0.6484 - val_accuracy: 0.6209

Epoch 00003: val_loss improved from 0.65919 to 0.64842, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 11s - loss: 0.7220 - accuracy: 0.6130 - val_loss: 0.6411 - val_accuracy: 0.6375

Epoch 00004: val_loss improved from 0.64842 to 0.64108, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 11s - loss: 0.7140 - accuracy: 0.6239 - val_loss: 0.6381 - val_accuracy: 0.6410

Epoch 00005: val_loss improved from 0.64108 to 0.63805, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 11s - loss: 0.7043

In [14]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([0, 0, 0, ..., 0, 1, 0]),
 0.7046875,
 0.7183308494783904,
 array([[1680,  664],
        [ 848, 1928]]),
 <keras.engine.functional.Functional at 0x7f258033d3d0>)